In [4]:
#fit ten different models 3 logistic regressors, 3 decision trees, 2 knns
#and 2 rfc the saw how they did in a voting classifier
import pandas as pd
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from sklearn.preprocessing import PolynomialFeatures,StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.neighbors import KNeighborsClassifier
pd.set_option('display.max_columns', 500)
import pickle as pkl

In [5]:
df = pd.read_csv('prepeddata.csv',index_col='Unnamed: 0')
df['stimulant'] = df['stimulant'].apply(lambda x: 'None' if x == 0 else 'low' if x <= 2 else 'high')
df


,Age,Education,Nscore,Escore,Oscore,Ascore,Cscore,Impulsive,SensationSeeking,Alchol,Caff,Chocalate,Nicotine,stimulant,depressant,hallucinagen
1,3,1,39.0,36.0,42.0,37.0,42.0,4.0,3.0,5,6,5,2,low,1.0,0.000000
2,2,4,29.0,52.0,55.0,48.0,41.0,3.0,6.0,5,6,6,4,low,1.0,2.000000
3,3,1,31.0,45.0,40.0,32.0,34.0,2.0,8.0,6,6,4,0,None,0.0,0.333333
4,1,3,34.0,34.0,46.0,47.0,46.0,2.0,3.0,4,5,4,2,low,1.5,0.000000
5,3,4,43.0,28.0,43.0,41.0,50.0,4.0,6.0,4,6,6,2,low,0.5,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1884,1,0,25.0,51.0,57.0,48.0,33.0,7.0,11.0,5,4,4,0,None,0.0,1.000000
1885,1,0,33.0,51.0,50.0,48.0,30.0,7.0,9.0,5,5,4,5,None,0.0,3.666667
1886,2,2,47.0,30.0,37.0,31.0,31.0,6.0,5.0,4,6,6,6,high,5.0,2.666667
1887,1,0,45.0,26.0,48.0,32.0,22.0,8.0,10.0,5,6,5,4,None,0.0,3.000000


In [6]:
stimtestdata = df.drop(['hallucinagen','depressant'],axis = 1)
stimx = stimtestdata.drop('stimulant',axis = 1)
stimy = stimtestdata.stimulant
poly = PolynomialFeatures(degree=2,interaction_only=True)
polystimtestdata = poly.fit_transform(stimx)
polystimtestdata = SelectKBest(k=15).fit_transform(polystimtestdata,stimy)



/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [0] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx,
/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [7]:
stimXtrain,stimXtest,stimYtrain,stimYtest = train_test_split(polystimtestdata,stimy)

In [8]:
stimscaler = StandardScaler()
stimXtrain = stimscaler.fit_transform(stimXtrain)
stimXtest = stimscaler.transform(stimXtest)
stimYtrain

887      low
909      low
3       None
1426    high
1167     low
        ... 
326     None
733      low
1463     low
1304    high
792      low
Name: stimulant, Length: 1413, dtype: object

Logistic regression classifiers

In [9]:
mod = LogisticRegression(class_weight = 'balanced'
                         ,max_iter=1000,solver='liblinear',warm_start=True,tol=0.2)
mod.fit(stimXtrain,stimYtrain)
stimtrianlogypred = mod.predict(stimXtrain)
stimtestlogypred = mod.predict(stimXtest)

In [10]:
f1_score(stimYtrain,stimtrianlogypred,average='weighted'),f1_score(stimYtest,stimtestlogypred,average='weighted')

(0.6000666854515404, 0.6063350456901512)

In [11]:
mod2 = LogisticRegression(class_weight='balanced',max_iter=4000,solver='sag',warm_start=True,tol=0.1)
mod2.fit(stimXtrain,stimYtrain)
stimtrianlog2ypred = mod2.predict(stimXtrain)
stimtestlog2ypred = mod2.predict(stimXtest)

In [12]:
f1_score(stimYtrain,stimtrianlog2ypred,average='weighted'),f1_score(stimYtest,stimtestlog2ypred,average='weighted')

(0.5036844176727114, 0.585072140269151)

In [13]:
mod3 = LogisticRegression(class_weight='balanced',max_iter=4000,solver='liblinear')
mod3.fit(stimXtrain,stimYtrain)
stimtrianlog3ypred = mod3.predict(stimXtrain)
stimtestlog3ypred = mod3.predict(stimXtest)

In [14]:
f1_score(stimYtrain,stimtrianlog3ypred,average='weighted'),f1_score(stimYtest,stimtestlog3ypred,average='weighted')

(0.6045825404903765, 0.6046000709352349)

Decision Trees

In [22]:
dtc = DecisionTreeClassifier(criterion='gini',ccp_alpha=0.0025)
dtc.fit(stimXtrain,stimYtrain)
dtcytrainpred = dtc.predict(stimXtrain)
dtcytestpred = dtc.predict(stimXtest)

In [23]:
f1_score(stimYtrain,dtcytrainpred,average='weighted'),f1_score(stimYtest,dtcytestpred,average='weighted')




(0.6439696149544933, 0.5645782224817902)

In [808]:
dtc2 = DecisionTreeClassifier(criterion='entropy',max_depth=50,ccp_alpha=0.0052)
dtc2.fit(stimXtrain,stimYtrain)
dtc2ytrainpred = dtc2.predict(stimXtrain)
dtc2ytestpred = dtc2.predict(stimXtest)

In [809]:
f1_score(stimYtrain,dtc2ytrainpred,average='weighted'),f1_score(stimYtest,dtc2ytestpred,average='weighted')


(0.6364909451219072, 0.5961935292565886)

In [810]:
dtc3 = DecisionTreeClassifier(class_weight='balanced',criterion='gini',ccp_alpha=0.005)
dtc3.fit(stimXtrain,stimYtrain)
dtc3ytrainpred = dtc3.predict(stimXtrain)
dtc3ytestpred = dtc3.predict(stimXtest)

In [811]:
f1_score(stimYtrain,dtc3ytrainpred,average='weighted'),f1_score(stimYtest,dtc3ytestpred,average='weighted')


(0.563192257910841, 0.559713462086882)

Knearest

In [812]:
knn = KNeighborsClassifier(n_neighbors=43,weights='uniform')
knn.fit(stimXtrain,stimYtrain)
knntrainypred=knn.predict(stimXtrain)
knntestypred=knn.predict(stimXtest)
f1_score(stimYtrain,knntrainypred,average='weighted'),f1_score(stimYtest,knntestypred,average='weighted')

(0.5903372146542069, 0.6223573911050958)

In [813]:
knn2 = KNeighborsClassifier(n_neighbors=11,weights='uniform')
knn2.fit(stimXtrain,stimYtrain)
knn2trainypred=knn2.predict(stimXtrain)
knn2testypred=knn2.predict(stimXtest)
f1_score(stimYtrain,knn2trainypred,average='weighted'),f1_score(stimYtest,knn2testypred,average='weighted')

(0.6416621363000531, 0.6010752368382112)

Random Forest

In [814]:
rfc = RandomForestClassifier(criterion='gini',ccp_alpha=0.005,n_estimators=1000,max_features=6,class_weight='balanced',max_depth=6,oob_score=True)
rfc.fit(stimXtrain,stimYtrain)

RandomForestClassifier(ccp_alpha=0.005, class_weight='balanced', max_depth=6,
                       max_features=6, n_estimators=1000, oob_score=True)

In [815]:
rfctrainypred = rfc.predict(stimXtrain)
rfctestypred = rfc.predict(stimXtest)
f1_score(rfctrainypred,stimYtrain,average='weighted'),f1_score(rfctestypred,stimYtest,average='weighted')

(0.5975396481890327, 0.5585939163880324)

In [816]:
rfc2 = RandomForestClassifier(criterion='entropy',ccp_alpha=0.01,n_estimators=10000,class_weight='balanced',oob_score=True)
rfc2.fit(stimXtrain,stimYtrain)

RandomForestClassifier(ccp_alpha=0.01, class_weight='balanced',
                       criterion='entropy', n_estimators=10000, oob_score=True)

In [817]:
rfc2trainypred = rfc2.predict(stimXtrain)
rfc2testypred = rfc2.predict(stimXtest)
f1_score(rfc2trainypred,stimYtrain,average='weighted'),f1_score(rfc2testypred,stimYtest,average='weighted')

(0.616206065643613, 0.5634420276292276)

voting classifier

In [818]:
vtc = VotingClassifier([('dtc2',dtc2),('rfc2',rfc2),('knn',knn),('knn2',knn2),('log',mod),('log3',mod3),('log2',mod2),('dtc',dtc),('dtc3',dtc3),("rfc",rfc)],voting='hard',verbose = True,n_jobs=-1)
vtc.fit(stimXtrain,stimYtrain)

VotingClassifier(estimators=[('dtc2',
                              DecisionTreeClassifier(ccp_alpha=0.0052,
                                                     criterion='entropy',
                                                     max_depth=50)),
                             ('rfc2',
                              RandomForestClassifier(ccp_alpha=0.01,
                                                     class_weight='balanced',
                                                     criterion='entropy',
                                                     n_estimators=10000,
                                                     oob_score=True)),
                             ('knn', KNeighborsClassifier(n_neighbors=43)),
                             ('knn2', KNeighborsClassifier(n_neighbors=11)),
                             ('log',
                              LogisticRegres...
                              LogisticRegression(class_weight='balanced',
                                 

In [819]:
vtcypred = vtc.predict(stimXtest)
vtcytrain = vtc.predict(stimXtrain)

print(f1_score(vtcytrain,stimYtrain,average='weighted'),f1_score(stimYtest,vtcypred,average='weighted'))


0.6142763019898725 0.6106521223598452


In [822]:
pkl.dump(vtc,open('vtc.pkl','wb'))